In [41]:
from pandas.core.common import flatten
import time
import sys
import os
import numpy as np
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.distance import add_hydrogen_bond_interactions, add_peptide_bonds
from functools import partial
#from graphein.protein.visualisation import plotly_protein_structure_graph

from graphein.protein.edges.distance import (add_peptide_bonds,
                                             add_k_nn_edges,
                                             add_distance_threshold,
                                             add_hydrogen_bond_interactions,
                                             add_disulfide_interactions,
                                             add_ionic_interactions,
                                             add_aromatic_interactions,
                                             add_aromatic_sulphur_interactions,
                                             add_cation_pi_interactions,
                                             add_delaunay_triangulation
                                            )

from graphein.protein.features.nodes import secondary_structure
from graphein.protein.subgraphs import extract_subgraph_from_secondary_structure
from graphein.protein.config import DSSPConfig
from graphein.protein.subgraphs import extract_surface_subgraph
from graphein.protein.features.nodes import rsa
import networkx as nx                                       
import logging
import matplotlib.pyplot as plt
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("graphein").setLevel(logging.INFO)
import graphein.protein as gp
from graphein.protein.edges.atomic import add_atomic_edges, add_bond_order, add_ring_status

In [ ]:
"""
params_to_change1 = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges]}
params_to_change2 = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges, add_bond_order, add_ring_status]}
params_to_change3 = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges, add_bond_order, partial(add_delaunay_triangulation)]}
params_to_change4 = {
    "granularity": "atom",
    "edge_construction_functions": [add_atomic_edges,
                                    add_bond_order,
                                    partial(add_delaunay_triangulation, allowable_nodes=["CA"])
                                    ]
    }
config = ProteinGraphConfig(**params_to_change)
"""

In [42]:
# define different graph
edge_funcs1 = {"edge_construction_functions": [partial(add_distance_threshold, long_interaction_threshold=5, threshold=10.),
                                                  add_aromatic_interactions,
                                                  add_hydrogen_bond_interactions,
                                                  add_disulfide_interactions,
                                                  add_ionic_interactions,
                                                  add_aromatic_sulphur_interactions,
                                                  add_cation_pi_interactions
                                                   ]
                 }




edge_funcs2 = {"edge_construction_functions": [
                                                  add_aromatic_interactions,
                                                  add_hydrogen_bond_interactions,
                                                  add_disulfide_interactions,
                                                  add_ionic_interactions,
                                                  add_aromatic_sulphur_interactions,
                                                  add_cation_pi_interactions
                                                   ]
                 }

edge_funcs3 = {"edge_construction_functions": [partial(add_distance_threshold, long_interaction_threshold=5, threshold=10.),
                                                  
                                                   ]
                 }

edge_funcs4 = {"edge_construction_functions": [add_delaunay_triangulation        
                                                   ]
                 }

edge_funcs5 = {"edge_construction_functions": [partial(add_k_nn_edges, k=3, long_interaction_threshold=0)]
                 }

edge_funcs = [edge_funcs1,edge_funcs2,edge_funcs3,edge_funcs4,edge_funcs5]

In [43]:
config = ProteinGraphConfig()
config.dict()

{'granularity': 'CA',
 'keep_hets': [],
 'insertions': False,
 'pdb_dir': None,
 'verbose': False,
 'exclude_waters': True,
 'deprotonate': False,
 'protein_df_processing_functions': None,
 'edge_construction_functions': [<function graphein.protein.edges.distance.add_peptide_bonds(G: 'nx.Graph') -> 'nx.Graph'>],
 'node_metadata_functions': [<function graphein.protein.features.nodes.amino_acid.meiler_embedding(n, d, return_array: bool = False) -> Union[pandas.core.series.Series, <built-in function array>]>],
 'edge_metadata_functions': None,
 'graph_metadata_functions': None,
 'get_contacts_config': None,
 'dssp_config': None}

In [44]:
# data load

pdb_p_GH30 = "/Users/eth-may/Desktop/dtu/semester6/graph_similarity/PDB/Q09DH4/Q09DH4.pdb"
pdb_p_GH35 = "/Users/eth-may/Desktop/dtu/semester6/graph_similarity/PDB/C3VDH8/C3VDH8.pdb"
pdb_p_GH16 = "/Users/eth-may/Desktop/dtu/semester6/graph_similarity/PDB/Q21N15/Q21N15.pdb"
pdb_p_GH27 = "/Users/eth-may/Desktop/dtu/semester6/graph_similarity/PDB/A0A6M4XDE9/A0A6M4XDE9.pdb"


In [88]:
import networkx as nx
from graphein.protein.graphs import construct_graph
from graphein.protein.subgraphs import extract_subgraph
def nodes_list(g,s,e):
    nodes =[]
    for i in g.nodes:
        nodes.append(i)
    return nodes[s-1:e] 

def get_network(s_g):
    nodes = []
    for i in s_g.nodes:
        nodes.append(i)
    
    edges = [] 
    for j in s_g.edges:
        edges.append(j)
    G = nx.Graph()
    
    for node in nodes:
        G.add_node(node)

    for link in edges:
        G.add_edge(link[0], link[1])
    return G

def sub_graph(edge_func):
    config = ProteinGraphConfig(**edge_func)

    g1 = construct_graph(config=config, pdb_path=pdb_p_GH30)
    g2 = construct_graph(config=config, pdb_path=pdb_p_GH35)
    g3 = construct_graph(config=config, pdb_path=pdb_p_GH16)
    g4 = construct_graph(config=config, pdb_path=pdb_p_GH27)

    s_g1 = extract_subgraph(g1, node_list=nodes_list(g1,33,478), inverse=False)
    s_g2 = extract_subgraph(g2, node_list=nodes_list(g2,20,408), inverse=False)
    s_g3 = extract_subgraph(g3, node_list=nodes_list(g3,328,567), inverse=False)
    s_g4 = extract_subgraph(g4, node_list=nodes_list(g4,48,380), inverse=False)
    
    G1 = get_network(s_g1)
    G2 = get_network(s_g2)
    G3 = get_network(s_g3)
    G4 = get_network(s_g4)
    
    return G1,G2,G3,G4

In [ ]:
import time
def GED(g1,g2):
    return nx.graph_edit_distance(g1, g2)

scores = []

for i in edge_funcs:
    G1,G2,G3,G4 = sub_graph(i)
    
    start = time.time()
    scores.append([GED(G1,G1),GED(G1,G2),GED(G1,G3),GED(G1,G4)])
    end = time.time()
    print(end - start)
print(scores)
#GED(G1,G2)



Output()

/Users/eth-may/opt/anaconda3/lib/python3.8/site-packages/biopandas/pdb/pandas_pdb.py:681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idxs["end_idx"] = ends.line_idx.values
Reading PDB file... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Output()

/Users/eth-may/opt/anaconda3/lib/python3.8/site-packages/biopandas/pdb/pandas_pdb.py:681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idxs["end_idx"] = ends.line_idx.values
Reading PDB file... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Output()

/Users/eth-may/opt/anaconda3/lib/python3.8/site-packages/biopandas/pdb/pandas_pdb.py:681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idxs["end_idx"] = ends.line_idx.values
Reading PDB file... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Output()

/Users/eth-may/opt/anaconda3/lib/python3.8/site-packages/biopandas/pdb/pandas_pdb.py:681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idxs["end_idx"] = ends.line_idx.values
Reading PDB file... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

In [ ]:
scores

In [ ]:
import nx.algorithms.isomorphism as iso
def iso_graph(g1,g2)
    gm = iso.GraphMatcher(g1, g2)
    mcs = gm.subgraph_is_isomorphic()

scores_2 = []
start = time.time()
for i in edge_funcs:
    G1,G2,G3,G4 = sub_graph(i)
    scores_2.append([iso_graph(G1,G1),iso_graph(G1,G2),iso_graph(G1,G3),iso_graph(G1,G4)])

end = time.time()
print(end - start)
print(scores_2)


[3341.0, 3346.0, 3344.0]


In [ ]:
#data load
GH16 = ['A0A7Z9D2A7','A5FC91','D9TGW6','E4N879','G8JN03','Q21N15','Q47XT7']
GH27 = ['A0A385TLX0','A0A3G3K5N5','A0A4P7NIG7','A0A6B8RPE7','A0A6M4XDE9','A0A8X8LEQ7']
GH30 = ['A0A3G2G2Q1','A0A6I5ZWY8','A0A7G8UJ47','A0A7U4Z621','H6NC90','Q09DH4']
GH35 = ['A0A0H3D6J3','A0A223HUM7','A0A4Y1V9N2','A0A7H5K301','F0VA11']

